In [2]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 86.7 MB/s eta 0:00:00


In [1]:
!pip install transformers accelerate sentencepiece


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
df = pd.read_excel('1_dataset_chatbot.xlsx')

In [5]:
df.head()

,nombre_cancion,estrofa
0,26 de mayo,el veintiseis del mes de mayo nacio un niñito ...
1,26 de mayo,"en carrizal tierra de poetas cerca del pueblo,..."
2,26 de mayo,entre la junta y patillal sobre lomas y sabana...
3,26 de mayo,un acordeon fue el gran encanto de aquel niñit...
4,26 de mayo,tarde a la casa llegaba y me sentaba en seguid...


In [6]:
def chunk_song_lines(lines, n_lines=4):
    chunks = []
    for i in range(0, len(lines), n_lines):
        chunk_text = " ".join(lines[i:i+n_lines]).strip()
        if chunk_text:
            chunks.append(chunk_text)
    return chunks


In [8]:
all_chunks = []

for song_name, group in df.groupby("nombre_cancion"):
    lines = group["estrofa"].tolist()
    chunks = chunk_song_lines(lines, n_lines=4)

    for idx, chunk in enumerate(chunks):
        all_chunks.append({
            "song": song_name,
            "chunk_id": idx,
            "text": chunk
        })


In [9]:
df_chunks = pd.DataFrame(all_chunks)
df_chunks.head()


,song,chunk_id,text
0,26 de mayo,0,el veintiseis del mes de mayo nacio un niñito ...
1,26 de mayo,1,tarde a la casa llegaba y me sentaba en seguid...
2,A Un Ladito Del Camino,0,hoy me siento enamorado por eso he venido a ve...
3,A Un Ladito Del Camino,1,"a orillitas del camino,"
4,A mi papá,0,voy a compone un merengue pa' cantarselo a pap...


In [10]:
encoder_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
texts = df_chunks["text"].tolist()
embeddings = encoder_model.encode(texts, show_progress_bar=True)
embeddings = embeddings.astype("float32")  # FAISS requiere float32


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
dim = embeddings.shape[1]  # dimensión del embedding
index = faiss.IndexFlatL2(dim)  # L2 = distancia euclidiana

# Agregar embeddings al índice
index.add(embeddings)

print("Total de vectores en FAISS:", index.ntotal)


Total de vectores en FAISS: 619


In [13]:
faiss.write_index(index, "songs_index.faiss")
df_chunks.to_pickle("songs_chunks.pkl")


In [14]:
def search_song(query, k=5):
    # 1. Embed de la consulta (USAR el modelo de embeddings, NO el LLM)
    query_emb = encoder_model.encode([query]).astype("float32")

    # 2. Buscar en FAISS
    distances, indices = index.search(query_emb, k)

    # 3. Recuperar los chunks
    results = df_chunks.iloc[indices[0]].copy()
    results["distance"] = distances[0]

    return results


In [15]:
search_song("¿De qué habla la canción sobre caminar solo?")


,song,chunk_id,text,distance
52,Camina,0,"camina, camina que caminando es cuando uno con...",0.531136
294,La Que Quiera Irse,0,y la que quiera irse que coga el camino y que ...,0.618297
337,La vecina,1,pero como soy pobre claro que yo no puedo hace...,0.618650
144,El Profeta,3,pero todo se acabo tu elegiste otro camino y e...,0.656199
66,Color de rosa,1,sombras del viejo camino saquen de mi este tor...,0.666747


In [16]:
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [17]:
def generate_answer(prompt, max_tokens=400):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.9
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [18]:
system_prompt = """
Tú eres un asistente llamado “Cantor”, especializado en dar acompañamiento emocional inspirado en las canciones del dataset del usuario.

Siempre que respondas:

Usa el contenido recuperado por RAG (versos, temas, metáforas, sentimientos).

No des consejos clínicos ni diagnósticos.

Interpreta las letras como reflexiones, aprendizajes y emociones humanas.

Habla con cercanía, sensibilidad y un tono poético sencillo.

No inventes versos si no aparecen en el contexto.

Si la letra no habla del tema, da una respuesta general inspirada en el estilo del artista.

Siempre conecta la pregunta del usuario con una emoción presente en las canciones recuperadas.

🔊 Estilo del asistente

Habla como un narrador reflexivo, calmado y honesto.

Puedes usar expresiones del vallenato o del estilo del artista, pero sin exagerar.

En temas de amor o tristeza usa un tono cálido y humano.

Enfatiza la resiliencia, la nostalgia, el amor y la superación (temas comunes en las canciones).

📝 Ejemplos de respuestas correctas
❓ Pregunta:

“¿Qué debería hacer si estoy triste por una ruptura amorosa?”

✔️ Respuesta esperada:

“En las canciones que encontré, hay un mensaje repetido: el dolor llega, pero también enseña.
Hay versos donde el cantante acepta la herida, pero también se recuerda a sí mismo que el camino sigue.
Tómate el tiempo para sentir, igual que en la estrofa donde dice que el corazón se cansa pero no se rinde.
A veces no hay que forzar nada… solo dejar que la música y los días pongan cada cosa en su lugar.”
"""

In [19]:
def answer_question(question):
    # Recuperar versos relevantes
    retrieved = search_song(question, k=5)

    # Validación por si no hay resultados
    if retrieved.empty:
        return "Lo siento, no encontré canciones relacionadas con eso."

    context = "\n".join(retrieved["text"].tolist()[:5])

    # Prompt final (Estructura para Phi-2)
    prompt = f"""Instruct: {system_prompt}
Context: {context}
User: {question}
Output:"""

    # Mover los inputs a la GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generación
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id # Buena práctica para evitar warnings
    )

    # Decodificar solo la parte nueva (evita repetir el prompt en la respuesta)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Limpieza opcional para mostrar solo la respuesta del asistente si el modelo repite el prompt
    if "Output:" in generated_text:
        return generated_text.split("Output:")[-1].strip()
    return generated_text

In [20]:
print(answer_question("¿Qué debería hacer si estoy triste por una ruptura amorosa?"))

En las canciones que encontré, hay un mensaje repetido: el dolor llega, pero también enseña. Hay versos donde el cantante acepta la herida, pero también se recuerda a sí mismo que el camino sigue. Tómate el tiempo para sentir, igual que en la estrofa donde dice que el corazón se cansa pero no se rinde. A veces no hay que forzar nada… solo dejar que la música y los días pongan cada cosa en su lugar.

Translation: In the songs I found, there is a recurring message: pain comes, but it also teaches. There are verses where the singer accepts the hurt, but also remembers themselves. Take the time to feel, just like in the song where


In [21]:
import textwrap

def answer_question(question):
    # Búsqueda en la base de datos (RAG)
    retrieved = search_song(question, k=3)

    if retrieved.empty:
        return "No encontré canciones para responder eso."

    # Unimos el contexto
    context_text = "\n".join(retrieved["text"].tolist())

    # PROMPT BLINDADO EN ESPAÑOL
    # Usamos etiquetas claras para que Phi-2 sepa qué es qué.
    # Le decimos explícitamente "Responde en Español".
    prompt = f"""Instruct: Eres un experto en música latina. Tu tarea es responder a la pregunta del usuario basándote ÚNICAMENTE en la letra de canción proporcionada abajo.
REGLAS:
1. Responde SIEMPRE en español.
2. No inventes información. Usa solo el contexto.
3. Si la letra habla de dolor, sé empático.
4. Sé breve y directo.

Contexto (Letra de canción):
{context_text}

Pregunta del Usuario:
{question}

Output:"""

    # Tokenización y GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generación con "Baja Temperatura"
    # Temperature 0.1 hace al modelo muy "robótico" y fiel a las instrucciones,
    # evitando que se invente historias de ingenieros.
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.1,       # <--- CAMBIO CLAVE: Muy bajo para evitar locuras
        top_p=0.90,
        repetition_penalty=1.3, # <--- CAMBIO CLAVE: Penaliza repetir instrucciones
        pad_token_id=tokenizer.eos_token_id
    )

    # Limpieza de la respuesta
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Cortar para obtener solo lo que sigue a "Output:"
    if "Output:" in generated_text:
        final_answer = generated_text.split("Output:")[-1].strip()
    else:
        final_answer = generated_text

    return textwrap.fill(final_answer, width=80)

# --- PRUEBA ---
print("Respuesta del modelo:")
print(answer_question("¿Qué debería hacer si la vida es estable todo el tiempo?"))

Respuesta del modelo:
Una posible respuesta al pregunta del usuario se podrían ser: "Si la vida fuera
estabelecerse para todos sin importar nuestros problemas ni nos daño porque
creen que existe un sentimiento universal entenderme"  OUTPUT: A possible
response could be: If life were stable all the time without any problems or harm
because we believe in something that understands and accepts you for who you
are. However, as someone with limited resources I cannot make it happen but
maybe if given your heart my love will always remain strong even though there
may still be wounds on my soul like those found in fairy tales very happy two of
them would live
